In [101]:
import pandas as pd
import shutil
from os import listdir, mkdir, path
from pathlib import Path
import sys
from email.parser import HeaderParser
import collections
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import time
from collections import Counter
import mailbox

In [102]:
columns = [ 
    'received1',
    'received2',
    'received3',
    'received4',
    'received5',
    'received6',
    'received7',
    'received8',
    'hops',
    'subject',
    'date',
    'message-id',
    'from',
    'return-path',
    'to',
    'content-type',
    'mime-version',
    'x-mailer',
    'content-transfer-encoding',
    'x-mimeole',
    'x-priority',
    'list-id',
    'lines',
    'x-virus-scanned',
    'status',
    'content-length',
    'precedence',
    'delivered-to',
    'list-unsubscribe',
    'list-subscribe',
    'list-post',
    'list-help',
    'x-msmail-priority',
    'x-spam-status',
    'sender',
    'errors-to',
    'x-beenthere',
    'list-archive',
    'reply-to',
    'x-mailman-version',
    'x-miltered',
    'x-uuid',
    'x-virus-status',
    'x-spam-level',
    'x-spam-checker-version',
    'references',
    'in-reply-to',
    'user-agent',
    'thread-index',
    'cc',
    'received-spf',
    'x-original-to',
    'content-disposition',
    'mailing-list',
    'x-spam-check-by',
    'domainkey-signature',
    'importance',
    'x-mailing-list',
    'body',
    'label'
]



In [103]:
def addEmailsToDict(email_list):
    list_of_rows = []
    # The label for phishing, set to '2'
    label = 2
    
    # Read the full email content
    for email in email_list:
        row_dict = {}
        # Parse recieved field
        received_list = email.get_all('received')
        hops = 0
        if received_list is not None:
            hops = len(received_list)
            col_name_recieved = 'received'

            for inx, received_field in enumerate(received_list):
                col = col_name_recieved + str(inx+1)
                row_dict[col] = received_field


        # Make everything lowercase to avoid issues
        features_lower_case = [x.lower() for x in email.keys()]

        # Parse everything else
        new_row = dict(zip(features_lower_case, email.values()))
        new_row['hops'] = hops


        for key,value in new_row.items():
            if key in columns:
                row_dict['label'] = label
                row_dict[key] = value


        list_of_rows.append(row_dict)
    return list_of_rows


In [107]:
def main(direcrotry, files):
    for file in files:
        mboxfile = mailbox.mbox(direcrotry + file)
        list_of_rows = addEmailsToDict(mboxfile)
        
        df = pd.DataFrame(list_of_rows)
        final_columns = []
        for col in columns:
            if col in df.columns:
                final_columns.append(col)

        df = df[final_columns]

        # Output the dataframe to a .csv file
        df.to_csv(direcrotry + 'csv/' + file.split('.')[0] + '.csv', index=False)


In [108]:
directory = '../datasets/nazario-phishing-corpus/'
files = [x for x in listdir(directory) if not x.startswith('.') and path.isfile(directory+x) ]
main(directory, files)